In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [7]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  13.88 MiB
  allocs estimate:  217596
  --------------
  minimum time:     15.947 ms (0.00% GC)
  median time:      17.621 ms (0.00% GC)
  mean time:        20.171 ms (14.40% GC)
  maximum time:     32.394 ms (42.53% GC)
  --------------
  samples:          249
  evals/sample:     1

In [8]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=1.0996270852667458, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=2.0996270852667456, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=3.099627085266745, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=4.099627085266745, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.099627085266744, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=6.099627085266746, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=7.099627085266746, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=8.099627085266748, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=9.099627085266743, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.099627085266746, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=11.099627085266745, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float6